### PINN approach for the 1-D duffing problem

$\ddot{x} + 2D\dot{x} + \omega_0^2(x + \mu x^3) = f(t)$

In [ ]:
import torch
import torchphysics as tp
import math
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp

In [ ]:
ode_params = {'D': .05,      # damping constant
              'c_nlin': 0.2  # nonlinearity
             }
sampling_frequency = 10 
end_t = 500 # sec
time = pd.to_timedelta(np.linspace(0, end_t, end_t*sampling_frequency + 1), unit="sec")

### ODE solution

In [ ]:
def duffing(t, y, D, c_nlin, F):
    """
    Duffing oscillator as descrbed in
    https://en.wikipedia.org/wiki/Duffing_equation

    Parameters
    ----------
    y : array of size 2
        initial conditions.
    t : float()
        time step at which the Force is computed.
    D : float()
        damping of the system.
    c_nlin : float()
        indicating the nonlinearity of the system.
    z : force applied
        force excitation at the mass.

    Returns
    -------
    derivatives of to duffing equation..

    """
    z1, z2 = y
    dz1dt = z2
    dz2dt = - 2 * D * z2 - z1 - c_nlin * z1**3 + F
    return(dz1dt, dz2dt)

In [ ]:
time.shape
F = np.random.randn(time.shape[0]) # some white noise data on right hand site
x0 = [0, 0] # initial conditions

In [ ]:
print(F.shape)
print(time.total_seconds().to_numpy().shape)

In [ ]:
numeric_solu = solve_ivp(duffing,
                         t_span=time.total_seconds().to_numpy(),
                         y0 = x0,
                         args=(ode_params['D'], ode_params['c_nlin'], F)
                        )